In [1]:
import pytesseract
from PIL import Image
import cv2
import numpy as np

In [67]:
file_path='E:\ML\OCR\img_set_2\img12.png' 
im = Image.open(file_path)
#im.show()
im.save('ocr.png', dpi=(300, 300))

In [68]:
image = cv2.imread('ocr.png')
image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
retval, threshold = cv2.threshold(image,127,255,cv2.THRESH_BINARY)

In [69]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [70]:
text = pytesseract.image_to_string(threshold)

In [71]:
print(text)

In [44]:
with open('E:\ML\OCR\output.txt', 'w',5 ,'utf-8') as text_file:
    text_file.write(text)

With additional preprocessing

In [1]:
import cv2
import pytesseract
import numpy as np
from pytesseract import Output

In [73]:
import cv2
import pytesseract
import numpy as np

# Set the path to your Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to preprocess the image
def preprocess_image(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to enhance text contrast
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 31, 11)

    # Apply blur to reduce noise
    blur = cv2.medianBlur(thresh, 3)

    # Sharpen the image to improve text edges
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharp = cv2.filter2D(blur, -1, kernel)

    return sharp

# Load the image
image = cv2.imread('E:\ML\OCR\img_set_2\img12.png')

# Resize the image for better large text recognition
scale_percent = 200  # Increase the scale for larger texts
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
resized_image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)

# Preprocess the resized image
preprocessed_image = preprocess_image(image)

# Perform OCR on the preprocessed image
text = pytesseract.image_to_string(preprocessed_image,lang='eng')

# Print the recognized text
print(text)